## CCI Tissue1

In [ ]:
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)

In [ ]:
library(CellChat)
library(patchwork)
options(stringsAsFactors = FALSE)

In [ ]:
object <- readRDS("./RDS/250422.combined_3.4_visium_labelupdate_tissue1.rds")

In [ ]:
spatial.locs = GetTissueCoordinates(object, scale = NULL, cols = c("imagerow", "imagecol"))

In [ ]:
scalefactors

In [ ]:
spatial.locs

scalefactors = jsonlite::fromJSON(txt = file.path("/data/User/revolvefire/projects/3.PPP_231118/231128_PPP/HN00218008_auto_result_10X/S24-6440/binned_outputs/square_008um/spatial/", 'scalefactors_json.json'))

spot.size = 8 
conversion.factor = spot.size/scalefactors$spot_diameter_fullres
spatial.factors = data.frame(ratio = conversion.factor, tol = spot.size/2)

library(dplyr)
spatial.locs <- spatial.locs %>% select(-cell)
print(spatial.locs)



In [ ]:
d.spatial <- computeCellDistance(coordinates = spatial.locs, ratio = spatial.factors$ratio, tol = spatial.factors$tol)
min(d.spatial[d.spatial!=0]) 

In [ ]:
object

In [ ]:
data.input = Seurat::GetAssayData(object, slot = "data", assay = "Spatial.008um") # normalized data matrix

In [ ]:
data.input

In [ ]:
meta = data.frame(labels = Seurat::Idents(object), samples = "sample1", row.names = names(Seurat::Idents(object))) 
meta$samples <- factor(meta$samples)
unique(meta$labels) 

In [ ]:
cellchat <- createCellChat(object = object, meta = meta, group.by = "labels",
                           datatype = "spatial", coordinates = spatial.locs, spatial.factors = spatial.factors)

In [ ]:
cellchat

In [ ]:
CellChatDB <- CellChatDB.human # use CellChatDB.human if running on human data
showDatabaseCategory(CellChatDB)

In [ ]:
CellChatDB.use <- subsetDB(CellChatDB, search = "Secreted Signaling", key = "annotation") # use Secreted Signaling

In [ ]:
# set the used database in the object
cellchat@DB <- CellChatDB.use

In [ ]:
library(future)

In [ ]:
plan(multisession, workers = 2)

In [ ]:
# subset the expression data of signaling genes for saving computation cost
cellchat <- subsetData(cellchat) # This step is necessary even if using the whole database
#future::plan("multisession", workers = 40) 
cellchat <- identifyOverExpressedGenes(cellchat)
cellchat <- identifyOverExpressedInteractions(cellchat, variable.both = F)
 
# project gene expression data onto PPI (Optional: when running it, USER should set `raw.use = FALSE` in the function `computeCommunProb()` in order to use the projected data)
# cellchat <- projectData(cellchat, PPI.mouse)
#execution.time = Sys.time() - ptm
#print(as.numeric(execution.time, units = "secs"))

In [ ]:
coords <- GetTissueCoordinates(object)     
min_dist <- min(dist(as.matrix(coords[, c("x", "y")])))

In [ ]:
min_dist

In [ ]:
scale.distance <- 1 / min_dist          
scale.distance

In [ ]:
scale.distance <- 2 / min_dist         
scale.distance

In [ ]:
#approximately 2um : scale.distance = 0.1

In [ ]:
cellchat <- computeCommunProb(cellchat, type = "truncatedMean", trim = 0.05,
                              distance.use = TRUE, interaction.range = 250, scale.distance = 0.1,
                              contact.dependent = TRUE, contact.range = 8) 
#> truncatedMean is used for calculating the average gene expression per cell group. 
#> [1] ">>> Run CellChat on spatial transcriptomics data using distances as constraints of the computed communication probability <<< [2024-02-22 10:39:52.098053]"
#> Molecules of the input L-R pairs are diffusible. Run CellChat in a diffusion manner based on the `interaction.range`.
#> [1] ">>> CellChat inference is done. Parameter values are stored in `object@options$parameter` <<< [2024-02-22 10:54:30.589965]"

In [ ]:
cellchat <- filterCommunication(cellchat, min.cells = 10)

In [ ]:
cellchat <- computeCommunProbPathway(cellchat)

In [ ]:
cellchat <- aggregateNet(cellchat)

#execution.time = Sys.time() - ptm
#print(as.numeric(execution.time, units = "secs"))
#> [1] 879.9437

In [ ]:
ptm = Sys.time()

groupSize <- as.numeric(table(cellchat@idents))
#par(mfrow = c(1,2), xpd=TRUE)
netVisual_circle(cellchat@net$count, vertex.weight = rowSums(cellchat@net$count), weight.scale = T, label.edge= F, title.name = "Number of interactions")
netVisual_circle(cellchat@net$weight, vertex.weight = rowSums(cellchat@net$weight), weight.scale = T, label.edge= F, title.name = "Interaction weights/strength")

In [ ]:
cellchat@netP$pathways

In [ ]:
netVisual_heatmap(cellchat, measure = "count", color.heatmap = "Blues", color.use=object@misc$use_color)
#> Do heatmap based on a single object

In [ ]:
netVisual_heatmap(cellchat, measure = "weight", color.heatmap = "Blues", color.use=object@misc$use_color)
#> Do heatmap based on a single object

In [ ]:
options(repr.plot.width=6, repr.plot.height=6, repr.plot.res = 300)

In [ ]:
pathways.show <- c("CCL") 
# Circle plot
par(mfrow=c(1,1), xpd = TRUE) # `xpd = TRUE` should be added to show the title
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle", pt.title=20, color.use=object@misc$use_color, vertex.label.cex = 0.7, , edge.width.max=10)

In [ ]:
netVisual_aggregate(
  cellchat,
  signaling        = pathways.show,
  layout           = "circle",
  pt.title         = 20,
  color.use        = object@misc$use_color,
  vertex.label.cex = 0.7, edge.width.max=10
)

dev.print(
  device = svglite,
  filename = "./figure4/250430_ccl_aggregate_circle.svg", 
  width    = 6,
  height   = 6
)

In [ ]:
options(repr.plot.width=10, repr.plot.height=10, repr.plot.res = 600)

In [ ]:
pathways.show <- c("CCL") 

In [ ]:
# Spatial plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, color.use =object@misc$use_color, signaling = pathways.show, layout = "spatial", edge.width.max = 5, vertex.size.max = 3, alpha.image = 0.15, vertex.label.cex = 5, point.size=0.1)

In [ ]:
p <- netVisual_aggregate(cellchat, color.use =object@misc$use_color, signaling = pathways.show, layout = "spatial", edge.width.max = 5, vertex.size.max = 3, alpha.image = 0.15, vertex.label.cex = 5, point.size=0.2)

In [ ]:
# Save the rotated image as an SVG file
ggsave("./figure4/250429_ccl_circle.svg", plot = p, device = "svg", width = 6, height = 6)

In [ ]:
options(repr.plot.width=6, repr.plot.height=6,  repr.plot.res = 300)

In [ ]:
pairLR.CCL <- extractEnrichedLR(cellchat, signaling = pathways.show, geneLR.return = FALSE)
LR.show <- pairLR.CCL[1,] # show one ligand-receptor pair
# Hierarchy plot
vertex.receiver = seq(1,4) # a numeric vector
netVisual_individual(cellchat, signaling = pathways.show,  pairLR.use = LR.show, vertex.receiver = vertex.receiver)
#> [[1]]
# Circle plot
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle", color.use =object@misc$use_color, vertex.label.cex = 0.7, edge.width.max=10)

In [ ]:
pairLR.CCL[1,]

In [ ]:
pairLR.CCL <- extractEnrichedLR(cellchat, signaling = pathways.show, geneLR.return = FALSE)
LR.show <- pairLR.CCL[2,] # show one ligand-receptor pair
# Hierarchy plot
vertex.receiver = seq(1,4) # a numeric vector
netVisual_individual(cellchat, signaling = pathways.show,  pairLR.use = LR.show, vertex.receiver = vertex.receiver)
#> [[1]]
# Circle plot
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle", color.use =object@misc$use_color, vertex.label.cex = 0.7)

In [ ]:
pairLR.CCL <- extractEnrichedLR(cellchat, signaling = pathways.show, geneLR.return = FALSE)
LR.show <- pairLR.CCL[3,] # show one ligand-receptor pair
# Hierarchy plot
vertex.receiver = seq(1,4) # a numeric vector
netVisual_individual(cellchat, signaling = pathways.show,  pairLR.use = LR.show, vertex.receiver = vertex.receiver)
#> [[1]]
# Circle plot
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle", color.use =object@misc$use_color)

In [ ]:
pathways.show <- c("CXCL") 
# Circle plot
par(mfrow=c(1,1), xpd = TRUE) # `xpd = TRUE` should be added to show the title
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle", pt.title=20)

In [ ]:
netVisual_aggregate(
  cellchat,
  signaling        = pathways.show,
  layout           = "circle",
  pt.title         = 20,
  color.use        = object@misc$use_color,
  vertex.label.cex = 0.7, edge.width.max=10
)

dev.print(
  device = svglite,
  filename = "./figure4/250430_cxcl_aggregate_circle.svg",
  width    = 6,
  height   = 6
)

In [ ]:
pairLR.CCL <- extractEnrichedLR(cellchat, signaling = pathways.show, geneLR.return = FALSE)
LR.show <- pairLR.CCL[1,] # show one ligand-receptor pair
# Hierarchy plot
vertex.receiver = seq(1,4) # a numeric vector
netVisual_individual(cellchat, signaling = pathways.show,  pairLR.use = LR.show, vertex.receiver = vertex.receiver)
#> [[1]]
# Circle plot
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle", color.use =object@misc$use_color, vertex.label.cex = 0.7)

In [ ]:
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle", color.use =object@misc$use_color, vertex.label.cex = 0.7, edge.width.max=10)

dev.print(
  device=svglite,
  filename = "./figure4/250429_cxcl1_ackr1_circle.svg",
  width    = 6,
  height   = 6
)

In [ ]:
pairLR.CCL <- extractEnrichedLR(cellchat, signaling = pathways.show, geneLR.return = FALSE)
LR.show <- pairLR.CCL[2,] # show one ligand-receptor pair
# Hierarchy plot
vertex.receiver = seq(1,4) # a numeric vector
netVisual_individual(cellchat, signaling = pathways.show,  pairLR.use = LR.show, vertex.receiver = vertex.receiver)
#> [[1]]
# Circle plot
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle", color.use =object@misc$use_color, vertex.label.cex = 0.7)

In [ ]:
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle", color.use =object@misc$use_color, vertex.label.cex = 0.7, edge.width.max=10)

dev.print(
  device=svglite,
  filename = "./figure4/250429_cxcl6_ackr1_circle.svg",
  width    = 6,
  height   = 6
)

In [ ]:
pairLR.CCL <- extractEnrichedLR(cellchat, signaling = pathways.show, geneLR.return = FALSE)
LR.show <- pairLR.CCL[3,] # show one ligand-receptor pair
# Hierarchy plot
vertex.receiver = seq(1,4) # a numeric vector
netVisual_individual(cellchat, signaling = pathways.show,  pairLR.use = LR.show, vertex.receiver = vertex.receiver)
#> [[1]]
# Circle plot
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle", color.use =object@misc$use_color, vertex.label.cex = 0.7)

In [ ]:
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle", color.use =object@misc$use_color, vertex.label.cex = 0.7, edge.width.max=10)

dev.print(
  device=svglite,
  filename = "./figure4/250429_cxcl8_ackr1_circle.svg",
  width    = 6,
  height   = 6
)

In [ ]:
pairLR.CCL <- extractEnrichedLR(cellchat, signaling = pathways.show, geneLR.return = FALSE)
LR.show <- pairLR.CCL[4,] # show one ligand-receptor pair
# Hierarchy plot
vertex.receiver = seq(1,4) # a numeric vector
netVisual_individual(cellchat, signaling = pathways.show,  pairLR.use = LR.show, vertex.receiver = vertex.receiver)
#> [[1]]
# Circle plot
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle", color.use =object@misc$use_color, vertex.label.cex = 0.7)

In [ ]:
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle", color.use =object@misc$use_color, vertex.label.cex = 0.7, edge.width.max=10)

dev.print(
  device=svglite,
  filename = "./figure4/250429_cxcl12_cxcr4_circle.svg",
  width    = 6,
  height   = 6
)

In [ ]:
pathways.show <- c("MIF") 
# Circle plot
par(mfrow=c(1,1), xpd = TRUE) # `xpd = TRUE` should be added to show the title
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle", pt.title=20)

In [ ]:
options(repr.plot.width=10, repr.plot.height=10, repr.plot.res = 600)

In [ ]:
# Spatial plot
par(mfrow=c(1,1))
netVisual_aggregate(cellchat, color.use =object@misc$use_color, signaling = pathways.show, layout = "spatial", edge.width.max = 5, vertex.size.max = 3, alpha.image = 0.25, vertex.label.cex = 5, point.size=0.2)

In [ ]:
pathways.show <- c("MIF") 
p <- netVisual_aggregate(cellchat, color.use =object@misc$use_color, signaling = pathways.show, layout = "spatial", edge.width.max = 5, vertex.size.max = 3, alpha.image = 0.15, vertex.label.cex = 5, point.size=0.2)

In [ ]:
# Save the rotated image as an SVG file
ggsave("./figure4/250429_MIF_cellchat.svg", plot = p, device = "svg", width = 10, height = 10)

In [ ]:
options(repr.plot.width=5, repr.plot.height=5,  repr.plot.res = 600)

In [ ]:
netAnalysis_contribution(cellchat, signaling = pathways.show, font.size=20)

In [ ]:
options(repr.plot.width=8, repr.plot.height=8,  repr.plot.res = 150)

In [ ]:
pairLR.CCL <- extractEnrichedLR(cellchat, signaling = pathways.show, geneLR.return = FALSE)
LR.show <- pairLR.CCL[1,] # show one ligand-receptor pair
# Hierarchy plot
vertex.receiver = seq(1,4) # a numeric vector
netVisual_individual(cellchat, signaling = pathways.show,  pairLR.use = LR.show, vertex.receiver = vertex.receiver)
#> [[1]]
# Circle plot
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle")

In [ ]:
pairLR.CCL <- extractEnrichedLR(cellchat, signaling = pathways.show, geneLR.return = FALSE)
LR.show <- pairLR.CCL[2,] # show one ligand-receptor pair
# Hierarchy plot
vertex.receiver = seq(1,4) # a numeric vector
netVisual_individual(cellchat, signaling = pathways.show,  pairLR.use = LR.show, vertex.receiver = vertex.receiver)
#> [[1]]
# Circle plot
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle")

In [ ]:
pairLR.CCL <- extractEnrichedLR(cellchat, signaling = pathways.show, geneLR.return = FALSE)
LR.show <- pairLR.CCL[3,] # show one ligand-receptor pair
# Hierarchy plot
vertex.receiver = seq(1,4) # a numeric vector
netVisual_individual(cellchat, signaling = pathways.show,  pairLR.use = LR.show, vertex.receiver = vertex.receiver)
#> [[1]]
# Circle plot
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle")

In [ ]:
pathways.show <- c("ANGPTL") 
# Circle plot
par(mfrow=c(1,1), xpd = TRUE) # `xpd = TRUE` should be added to show the title
netVisual_aggregate(cellchat, signaling = pathways.show, layout = "circle", pt.title=20)

In [ ]:
netVisual_aggregate(
  cellchat,
  signaling        = pathways.show,
  layout           = "circle",
  pt.title         = 20,
  color.use        = object@misc$use_color,
  vertex.label.cex = 0.7, edge.width.max=10
)

dev.print(
  device=svglite,
  filename = "./figure4/250429_angptl_aggregate_circle.svg",
  width    = 6,
  height   = 6
)

In [ ]:
pairLR.CCL <- extractEnrichedLR(cellchat, signaling = pathways.show, geneLR.return = FALSE)
LR.show <- pairLR.CCL[1,] # show one ligand-receptor pair
# Hierarchy plot
vertex.receiver = seq(1,4) # a numeric vector
netVisual_individual(cellchat, signaling = pathways.show,  pairLR.use = LR.show, vertex.receiver = vertex.receiver)
#> [[1]]
# Circle plot
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle")

In [ ]:
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle", color.use =object@misc$use_color, vertex.label.cex = 0.7, edge.width.max=10)

dev.print(
  svglite,
  filename = "./figure4/250429_angptl4_cdh5_circle.svg",
  width    = 6,
  height   = 6
)

In [ ]:
pairLR.CCL <- extractEnrichedLR(cellchat, signaling = pathways.show, geneLR.return = FALSE)
LR.show <- pairLR.CCL[2,] # show one ligand-receptor pair
# Hierarchy plot
vertex.receiver = seq(1,4) # a numeric vector
netVisual_individual(cellchat, signaling = pathways.show,  pairLR.use = LR.show, vertex.receiver = vertex.receiver)
#> [[1]]
# Circle plot
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle")

In [ ]:
pairLR.CCL <- extractEnrichedLR(cellchat, signaling = pathways.show, geneLR.return = FALSE)
LR.show <- pairLR.CCL[3,] # show one ligand-receptor pair
# Hierarchy plot
vertex.receiver = seq(1,4) # a numeric vector
netVisual_individual(cellchat, signaling = pathways.show,  pairLR.use = LR.show, vertex.receiver = vertex.receiver)
#> [[1]]
# Circle plot
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle")

In [ ]:
pairLR.CCL <- extractEnrichedLR(cellchat, signaling = pathways.show, geneLR.return = FALSE)
LR.show <- pairLR.CCL[4,] # show one ligand-receptor pair
# Hierarchy plot
vertex.receiver = seq(1,4) # a numeric vector
netVisual_individual(cellchat, signaling = pathways.show,  pairLR.use = LR.show, vertex.receiver = vertex.receiver)
#> [[1]]
# Circle plot
netVisual_individual(cellchat, signaling = pathways.show, pairLR.use = LR.show, layout = "circle")